In [1]:
import sys
import functools

In [2]:
class Node:
    def __init__(self, value: int, left: 'Node' = None, right: 'Node' = None):
        self.value = value
        self.left = left
        self.right = right

    def __repr__(self):
        return f'{self.value}: ({self.left}, {self.right})'

    def is_leaf(self) -> bool:
        return self.left is None and self.right is None

    @classmethod
    def from_int(cls, node: int) -> 'Node':
        return cls(node, None, None)

    def as_string(self) -> str:
        return str(self.value)


In [3]:
def load_input() -> list[Node]:
    with open('../../data/day11-input.txt') as f:
        return [Node.from_int(int(x)) for x in f.read().strip().split(' ')]


In [4]:
def apply_rules(node: Node) -> tuple[Node]|tuple[Node,Node]:
    """What does it become?"""
    if node.value == 0:
        return (Node.from_int(1),)
    else:
        as_string = node.as_string()
        if len(as_string) % 2 == 0:
            return (Node.from_int(int(as_string[:len(as_string) // 2])), Node.from_int(int(as_string[len(as_string) // 2:])))
        else:
            return (Node.from_int(2024*node.value),)

def process_input(blink_count: int = 0) -> list[Node]:
    nodes = load_input()

    for blink in range(blink_count):
        next_nodes = []
        for i, node in enumerate(nodes):
            result = apply_rules(node)
            if len(result) == 1:
                node.left = result[0]
                next_nodes.append(result[0])
            else:
                node.left = result[0]
                next_nodes.append(result[0])
                node.right = result[1]
                next_nodes.append(result[1])
        nodes = next_nodes
    return nodes

def traverse(node):
    results = []
    if node.is_leaf():
        return [node.value]
    else:
        if node.left:
            results = results + traverse(node.left)
        if node.right:
            results = results + traverse(node.right)
    return results

nodes = process_input(25)
counter = 0
for node in nodes:
    for i in traverse(node):
        counter += 1
print('counter =', counter)

counter = 207683


### Part 2


New strategy. I'm memoizing as much as I can. We are only looking for the count of stones.
I originally thought it'd be nice to build a tree so that I could see the ancestry, but
75 (even 25) iterations is just way too much. My PC maxed out 64 GB of ram and eventually crashed.


In [5]:
del apply_rules, process_input, node, nodes, load_input

In [6]:
def load_input() -> list[int]:
    with open('../../data/day11-input.txt') as f:
        return [int(x) for x in f.read().strip().split(' ')]

@functools.lru_cache(maxsize=None)
def get_tens():
    tens = []
    value = 10
    while value < sys.maxsize:
        tens.append(value)
        value *= 10
    return tens

@functools.lru_cache(maxsize=None)
def count_digits(num: int) -> int:
    tens = get_tens()
    for i, x in enumerate(tens):
        if num < x:
            return i+1
    raise ValueError("Unhandled value: {}".format(num))

@functools.lru_cache(maxsize=None)
def get_splitter(length: int):
    return 10**(length//2)

@functools.lru_cache(maxsize=None)
def split_even_length_number(num: int) -> tuple[int, int]:
    splitter = get_splitter(count_digits(num))
    return (num // splitter, num % splitter)


In [7]:

def apply_rules(stone: int) -> tuple[int]|tuple[int, int]:
    """What does it become?"""
    if stone == 0:
        return (1,)
    else:
        num_digits = count_digits(stone)
        if num_digits % 2 == 0:
            return split_even_length_number(stone)
        else:
            return (2024*stone,)

def process_input(blink_count: int) -> dict[int, int]:
    stones = {}
    for stone in load_input():
        if stone not in stones:
            stones[stone] = 1
        else:
            stones[stone] += 1

    for blink in range(blink_count):
        intermediates = {}
        for stone, count in stones.items():
            for rock in apply_rules(stone):
                if rock in intermediates:
                    intermediates[rock] += count
                else:
                    intermediates[rock] = count
        stones = intermediates

    return stones


assert 207683 == sum(process_input(25).values())

In [8]:
print(sum(process_input(75).values()))

244782991106220
